In [20]:
import gensim
import numpy as np

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
type(model.wv)

/Users/kenalo37/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


gensim.models.keyedvectors.Word2VecKeyedVectors

In [11]:
## prepare data for training

# read semi-clean data
import pandas as pd
doc_complete = pd.read_csv('~/NSV/Data/behavior.csv')
doc_complete = list(doc_complete.iloc[:,1])

doc = [d.split(' ') for d in doc_complete]
doc

[['', 'chinese', 'fire', 'drill', ''],
 ['abusive', 'cycle'],
 ['action', 'assembly', 'theory'],
 ['active'],
 ['adoption'],
 ['age', '17', '19'],
 ['aggresion', 'towards', 'objects'],
 ['aggressive'],
 ['alcohol', 'use'],
 ['alienation'],
 ['alloplastic', 'adaptation'],
 ['always', 'and', '', 'never', '', 'statements'],
 ['ambitious'],
 ['anger'],
 ['animal', 'abuse'],
 ['animal', 'spirits'],
 ['argumentative'],
 ['asccismus'],
 ['assault', 'with', 'gun'],
 ['assault', 'with', 'knife'],
 ['assertive'],
 ['attention', 'deficit', 'hyperactivity', 'disorder'],
 ['attention', 'seeking'],
 ['attitude', 'change'],
 ['autoplastic', 'defense', 'mechanism'],
 ['avoidance'],
 ['avoidance', 'coping'],
 ['bad', 'coping'],
 ['bad', 'habit'],
 ['baiting'],
 ['beat'],
 ['beat', 'up'],
 ['behavior'],
 ['behavior', 'change'],
 ['behavioral', 'confirmation'],
 ['behavioral', 'contagion'],
 ['behavioral', 'modernity'],
 ['behavioral', 'urbanism'],
 ['belittle', 'partner'],
 ['belittling', '', 'condescen

In [12]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Build Lemmatizer
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [17]:
def clean_lemmatize(doc):
    """lemmatize and clean doc"""
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# Run word_list into an input volume
def clean(word_list):
    lemmatized_list = []
    for word in word_list:
        #assert type(word) == list
        if type(word) == list:
            temp = [clean_lemmatize(x) for x in d]
            lemmatized_list.append(temp)
    for i in range(len(lemmatized_list)):
        temp = [x for x in lemmatized_list[i] if x!= '' and x.isalpha()]
        lemmatized_list[i] = temp

In [21]:
def input_volume(word_list, timesteps):
    clean(word_list)
    w2v = np.zeros([len(word_list), timesteps, 300])
    c1 = 0
    for word in word_list:
        x = np.zeros([6, 300])
        if type(word) == list:
            c2 = 0
            for w in word:
                try: 
                    x[c2] = model[w]
                    c2 += 1
                except KeyError:
                    pass
        elif type(word) == str:
            x[0] = model[word]
        w2v[c1] = x
        c1 += 1
    return w2v

In [22]:
input_vol = input_volume(doc, 6)

NameError: global name 'model' is not defined

In [60]:
np.save('input_vol', input_vol)